In [ ]:
from __future__ import division
import os
import math
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pylab
from matplotlib import rc
import pandas as pd
import scipy.stats as stats

from sklearn.decomposition import PCA
from IPython.html.widgets import interact
from IPython.html import widgets
from IPython.display import display

from clustering import get_data, hierarchical_clustering
from survival import survival_analysis
from clean_data import CANCER_TYPES

In [ ]:
can_types = []
for c in CANCER_TYPES:
    f1 = '../results/' + c + os.sep + 'silent_mutation_pathway_score.txt'
    f2 = '../results/' + c + os.sep + 'nsilent_mutation_pathway_score.txt'
    f3 = '../data/processed/' + c + os.sep + c + '_clinical.csv'

    if os.path.exists(f1) and os.path.exists(f2) and os.path.exists(f3):
        can_types.append(c)
    
print "There are %d cancer types ready to be analysed" % len(can_types)
can_type_wid = widgets.Dropdown(description="Select Cancer Type", options=can_types)
display(can_type_wid)



In [ ]:
k = 5
can = can_type_wid.value
(sig_nsdf, clinical_df) = get_data(can, 'mut', nsdf_norm_factor=1)
print sig_nsdf.shape
clinical_df = clinical_df.loc[clinical_df.time > 0]
common_samples = sig_nsdf.columns.intersection(clinical_df.index)
sig_nsdf = sig_nsdf[common_samples]
clinical_df = clinical_df.loc[common_samples]
df = sig_nsdf.transpose()

data_array = df[df.columns[:k]].values
labels = df.index
memb = hierarchical_clustering(data_array, labels)
print memb.value_counts()


In [ ]:
def KM(memb, clinical_df, clusters, drop=''):
    memb = memb.copy()
    clusts = [int(x) for x in str(clusters)]
    rest = list(set(memb.unique()).difference(clusts))

    if drop and drop > 0:    
        ignore = [int(x) for x in str(drop)]
        memb = memb[~memb.isin(ignore)]
    if drop and drop < 0:    
        ignore = rest
        memb = memb[~memb.isin(ignore)]


    memb[memb.isin(rest)] = rest[0]
#     print memb.value_counts()
    clinical_df = clinical_df.join(memb, how='inner', lsuffix='old')
    survival_analysis(clinical_df)
    print memb.value_counts()
    
    return memb, clinical_df

idx, cdf = KM(memb, clinical_df, 3)
# print memb.value_counts()

In [ ]:

import pathway_analysis as pathway

# Load preprocessed mutation data
data_dir = '../data/processed/' + can
# mut_categories = ['silent', 'nsilent']
mut_categories = ['nsilent']
path_df = pathway.preprocess_pathway_data()
mut_path = data_dir + os.sep + can_type + '_nsilent_mutation.txt'
mut_df = pd.read_table(mut_path, index_col=0, header=0)

# path_type_wid = widgets.Dropdown(description="Select Pathway", options=list(sig_nsdf.index[:k]))
# display(path_type_wid)



# Adopted from to OxanaSachenkova/hclust-python at github
def hierarchical_clustering(df, mut_df, path_df, labels='', color_list=None):
    data_array = df.transpose().values
    if type(labels) == str:
        labels = map(str, range(1, data_array.shape[0]+1))
    data_dist = pdist(data_array) # computing the distance
    Y = linkage(data_dist, method='single')

    x0 = 0.2; w=0.7; y00 = 0.93; y01 = 0.8; y = 25
    # Compute and plot first dendrogram.
    fig = plt.figure(figsize=(20,y))
    ax2 = fig.add_axes([x0,y00,w,0.06])
    Z2 = dendrogram(Y,labels=labels, leaf_font_size=8)#, color_list=color_list.values)
    idx2 = Z2['leaves']
    ax2.set_yticks([])
   

    #Compute and plot the heatmap
    axmatrix = fig.add_axes([x0,y01,w,0.1])
    D = df.loc[:,df.columns[idx2]].values
    im = axmatrix.matshow(D, aspect='auto', origin='upper', cmap=plt.cm.YlGnBu)
  
    axmatrix.set_yticklabels([''] + list(df.index), rotation=0)
    axmatrix.set_xticks([])

    
    # Plot colorbar.
    axcolor = fig.add_axes([0.91,y01,0.02,0.1])
    plt.colorbar(im, cax=axcolor)
    
    cmaps = ['Oranges', 'binary', 'Spectral']
    h = y01/df.shape[0] - df.shape[0]*0.01
    for i in range(df.shape[0]):
        y0 = y01-h*(i+1)
    
        # Plot individual mutation
        ax3 = fig.add_axes([x0,y0,w, h] )
        path_genes = path_df[df.index[i]].dropna()
        path_genes_df = mut_df.loc[path_genes, df.columns[idx2]].fillna(0)
        path_genes_df = path_genes_df.loc[path_genes_df.sum(axis=1) != 0, :]
        sorted_df = path_genes_df.sum(axis=1).order(ascending=False)
        path_genes_df = path_genes_df.loc[sorted_df.index]
#         print D.shape, path_genes_df.shape, df.index[i]
        D2 = path_genes_df.values
        im2 = ax3.matshow(D2, aspect='auto', origin='upper', cmap=cmaps[i%len(cmaps)])
        ax3.set_yticks(range(len(path_genes_df.index)))
        ax3.set_yticklabels(list(path_genes_df.index), fontsize=9)
        ax3.set_xticks([])
        # Plot colorbar.
        axcolor2 = fig.add_axes([0.91,y0,0.02,h])
        plt.colorbar(im2, cax=axcolor2)
        
#     print path_genes_df.head()
    plt.show()

    return idx2

# colors = ['g', 'b', 'r', 'c']
# cdf['colors'] = [colors[i-1] for i in cdf['groups']]

memb = hierarchical_clustering(sig_nsdf.head(), mut_df, path_df, labels)


In [ ]:
sig_nsdf.loc[:,sig_nsdf.columns[memb]].head()

In [ ]:
@interact(path_genes=widgets.Dropdown(description="Select Pathway", options=list(sig_nsdf.index[:k])))
def plot_matrix(path_genes):
    path_genes = path_df[path_genes].dropna()
    path_genes_df = mut_df.loc[path_genes, sig_nsdf.columns[memb]].fillna(0)
    path_genes_df.loc[path_genes_df.sum(axis=1) != 0, :]

    fig = plt.figure(figsize=(20,6))
    axmatrix = fig.add_axes([0,0,0.6,0.6])
    D = path_genes_df.values
    im = axmatrix.matshow(D, aspect='auto', origin='lower', cmap=plt.cm.YlGnBu)
    axmatrix.set_yticklabels([''] + list(path_genes_df.index), rotation=0)
    axmatrix.set_xticklabels(list(sig_nsdf.columns[memb]))
    plt.colorbar(im)
   
    plt.show()





In [ ]:
df = sig_nsdf.transpose()
print clinical_df['Auxiliary'].value_counts()
mss_df = clinical_df[clinical_df.Auxiliary == 'positive']
df = df.loc[df.index.intersection(mss_df.index)]
print df.shape


In [ ]:
X = sig_nsdf.transpose().values
pca = PCA(n_components=5)
X_red = pca.fit_transform(X)
print X_red.shape, sig_nsdf.shape
print pca.explained_variance_
print  np.cumsum(pca.explained_variance_ratio_)
memb = hierarchical_clustering(X_red)
print memb.value_counts()




In [ ]:
# http://en.wikipedia.org/wiki/DNA_codon_table
amino_acids = 'arndcqeghi'
amino_acids += 'lkmfpstwyv'
amino_acids += 'bo'
codons = '4622222423'
codons += '6212464124'
codons += '13'
spr = 1; nspr = 1;
spr =  sum([int(x)-1 for x in codons])/(22*27)
nspr = 1 - spr



In [ ]:
def preprocess_pathway_data():
    """Load GSEA MSigDB Broad Pathway DB"""
    input_dir = '../data/pathways'
    filename = input_dir + os.sep + 'kegg_biocarta_pid_positional.txt'
    pathways = {}
    with open(filename, 'r') as f:
        for line in f:
            p = line.strip().split('\t')
            pathways[p[0]] = p[2:]

    df = pd.DataFrame.from_dict(pathways, orient='index').transpose()

    return df

In [ ]:
df = preprocess_pathway_data()
df.count().describe()